In [1]:
import pandas as pd
import requests

from api_utils import loadCSVData,updateIncidenceTable,getPandaFrameForCountry
from api_utils import getRawDataToCSV
from api_utils import buildbrutIncidence,incidenceMovingAverage,incidenceExpMovingAverage
pd.options.mode.chained_assignment = None

import logging
logging.basicConfig(level=logging.DEBUG,
 format=' %(threadName)s -- %(levelname)s -- %(message)s')

In [2]:
# MODELING TESTS 
from modeling import continent_map, country_map, get_countrymap

#continent_map
#get_countrymap()
#country_map

In [3]:
rawdf=pd.DataFrame(getPandaFrameForCountry('Aruba'))
print(rawdf.head(1))

 MainThread -- DEBUG -- Starting new HTTPS connection (1): api.covid19api.com:443
 MainThread -- DEBUG -- https://api.covid19api.com:443 "GET /total/country/Aruba HTTP/1.1" 200 31
 MainThread -- DEBUG -- request OK for 200


Empty DataFrame
Columns: []
Index: []


In [5]:
# TEST ON getPandaFrameForCountry (FOR RESPONSES TIME AND CODES)
for i in country_map.keys():
    rawdf=pd.DataFrame(getPandaFrameForCountry(i))
    print(rawdf.head(1))

  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  France                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
   Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Germany                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
   Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Belgium                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0   Spain                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province C

          Country CountryCode Province City CityCode Lat Lon  Confirmed  \
0  United Kingdom                                      0   0          0   

   Deaths  Recovered  Active                  Date  
0       0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Sweden                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
       Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Switzerland                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Turkey                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
   Country

  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Brazil                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
    Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Colombia                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
     Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Argentina                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0    Peru                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Prov

   Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Bolivia                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
       Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Afghanistan                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
  Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0   India                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
     Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Indonesia                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
      Country Coun

              Country CountryCode Province City CityCode Lat Lon  Confirmed  \
0  Russian Federation                                      0   0          0   

   Deaths  Recovered  Active                  Date  
0       0          0       0  2020-01-22T00:00:00Z  
    Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Thailand                                      0   0          4       0   

   Recovered  Active                  Date  
0          2       2  2020-01-22T00:00:00Z  
      Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Uzbekistan                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00Z  
    Country CountryCode Province City CityCode Lat Lon  Confirmed  Deaths  \
0  Malaysia                                      0   0          0       0   

   Recovered  Active                  Date  
0          0       0  2020-01-22T00:00:00

KeyboardInterrupt: 

In [5]:
countrylist = ['France','United Kingdom','China']
seriesdict = {}
for elem in countrylist : 
    updateIncidenceTable(elem)
    seriesdict[elem] = loadCSVData(elem,'Inc')['Confirmed_eMMincidence']
    
print(seriesdict['France'])

 MainThread -- INFO -- entering updateIncidenceTable for France
 MainThread -- INFO -- , incidence update skipped for France : no recent data detected 
 MainThread -- INFO -- entering updateIncidenceTable for United Kingdom
 MainThread -- INFO -- , incidence update skipped for United Kingdom : no recent data detected 
 MainThread -- INFO -- entering updateIncidenceTable for China
 MainThread -- INFO -- , incidence update skipped for China : no recent data detected 


0          0.00
1          0.00
2          0.20
3          0.28
4          0.25
         ...   
358    16882.49
359    17361.54
360    15654.38
361    17829.45
362    16046.50
Name: Confirmed_eMMincidence, Length: 363, dtype: float64
